https://www.joinquant.com/view/user/floor?type=mainFloor

In [ ]:
# 导入函数库
from jqdata import *

def initialize(context):
    # 设定沪深300作为基准
    set_benchmark('000300.XSHG')
    # 开启动态复权模式(真实价格)
    set_option('use_real_price', True)
    # 股票类每笔交易时的手续费是：买入时佣金万分之三，卖出时佣金万分之三加千分之一印花税, 每笔交易佣金最低扣5块钱
    set_order_cost(OrderCost(close_tax=0.001, open_commission=0.0003, close_commission=0.0003, min_commission=5), type='stock')
    # 持股数量
    g.N = 10
    run_monthly(handle, 1)


def handle(context):
    stocks = get_index_stocks('000300.XSHG')
    # print(stocks)
    
    # 沪深300股票的过去30天(不包含今天)的每天的收盘价
    df_close = history(30, field='close', security_list=list(stocks)).T
    # 加入一列——每只股票这30天收益率
    df_close['ret'] = (df_close.iloc[:,-1]-df_close.iloc[:,0]) / df_close.iloc[:,0]
    # ascending=False:根据收益率降序排列——动量策略
    # ascending=True:根据收益率升序排列——反转策略
    sorted_stocks = df_close.sort_values('ret', ascending=False).index
    # 找到收益最大/最小的N只股票
    to_hold = sorted_stocks[:g.N]

    for stock in context.portfolio.positions:
        if stock not in to_hold:
            order_target_value(stock, 0)
            
    to_buy = [stock for stock in to_hold if stock not in context.portfolio.positions]
    
    if len(to_buy) > 0:
        cash_per_stock = context.portfolio.available_cash / len(to_buy)
        for stock in to_buy:
            order_value(stock, cash_per_stock)

In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

In [27]:
csitrade=pd.read_csv('D:/服务器/all/baselines/hs300_2018_2023.csv')
# csitrade.rename(columns={'Close':'close'},inplace=True)

In [28]:
#数据一定要按股票排序
csitrade.sort_values(['kdcode','dt'],inplace=True)
csitrade['close_r']=csitrade['close']/csitrade['close'].shift(1)

In [29]:
csitrade.reset_index(inplace=True,drop=True)

In [30]:
csitrade

,kdcode,dt,close,open,high,low,prev_close,adjfactor,turnover,volume,close_r
0,000001.SZ,2018-01-02,13.70,13.35,13.93,13.32,13.30,101.992244,2.856544e+09,208159255.0,NaN
1,000001.SZ,2018-01-03,13.33,13.73,13.86,13.20,13.70,101.992244,4.006221e+09,296249838.0,0.972993
2,000001.SZ,2018-01-04,13.25,13.32,13.37,13.13,13.33,101.992244,2.454544e+09,185450948.0,0.993998
3,000001.SZ,2018-01-05,13.30,13.21,13.35,13.15,13.25,101.992244,1.603290e+09,121031272.0,1.003774
4,000001.SZ,2018-01-08,12.96,13.25,13.29,12.86,13.30,101.992244,2.806099e+09,215862081.0,0.974436
...,...,...,...,...,...,...,...,...,...,...,...
437095,688981.SH,2023-12-25,53.01,54.60,55.00,52.85,54.79,1.000000,9.541097e+08,17788400.0,0.967512
437096,688981.SH,2023-12-26,51.85,53.01,53.23,51.80,53.01,1.000000,7.540936e+08,14401600.0,0.978117
437097,688981.SH,2023-12-27,51.91,51.83,52.29,51.18,51.85,1.000000,7.864514e+08,15173500.0,1.001157
437098,688981.SH,2023-12-28,53.13,51.70,53.45,51.68,51.91,1.000000,1.015243e+09,19211900.0,1.023502


In [31]:
df=csitrade
df['ret']=np.nan
df.rename(columns={'kdcode':'instrument','dt':'datetime'},inplace=True)
# df.rename(columns={'company':'instrument','Date':'datetime'},inplace=True)
df['datetime'] = df['datetime'].astype('datetime64[ns]')
date_unique=df['datetime'].unique()
ins_unique=df['instrument'].unique()

In [155]:
choice = 1
num = 16

In [164]:
if choice == 1:
    #动量策略
    dff_ret=pd.DataFrame()
    # 遍历股票
    for ins in ins_unique:
        # 选特点股票数据
        df_tic=df[df['instrument']==ins]
        df_tic.reset_index(inplace=True)
        # 股票数据每一行
        for i in range(0,len(df_tic)):
            # 23年以后
            if df_tic['datetime'][i]>datetime(2023,1,1):
                flag = 0
                date=df_tic['datetime'][i]
                # 找到当前日期的位置
                loc=np.where(date_unique==date)
                # 找到80天前的日期位置
                predate_loc=np.where(date_unique==date)[0][0]-60
                date_pre=date_unique[predate_loc]
                # 如果80天前的收盘价存在
                if len(df_tic.loc[df_tic['datetime']==date_pre]['close'].values):
                    x=df_tic.loc[df_tic['datetime']==date_pre]['close'].values[0]
                else:
                    flag = 1  # 标记为1，表示80天前的数据不存在
                
                # 找到前一天的日期位置
                yestdaydate_loc=np.where(date_unique==date)[0][0]-1
                date_yest=date_unique[yestdaydate_loc]
                if len(df_tic.loc[df_tic['datetime']==date_pre]['close'].values):
                    y=df_tic.loc[df_tic['datetime']==date_yest]['close'].values[0]
                else:
                    flag = 1
                if flag:
                    df_tic['ret'][i] = 0  # 如果数据不完整，则返回值设置为0
                    continue
                df_tic['ret'][i]=(y-x)/x  # 计算收益率
        df_tic.dropna(how='any',inplace=True)
        dff_ret=pd.concat([dff_ret,df_tic],axis=0)  # 将处理后的数据合并
    dff_ret.reset_index(inplace=True)
    ret_date_unique=dff_ret['datetime'].unique()  # 获取有收益数据的独特日期
    
    # 选择特定日期的股票，并计算最小和最大收益的股票标识符
    day=dff_ret[dff_ret['datetime']==np.datetime64('2023-01-03')]
    sma_ins=day.nsmallest(num,columns='ret').instrument
    larg_ins=day.nlargest(num,columns='ret').instrument
    
    df_return=pd.DataFrame()
    for date in ret_date_unique:   # 遍历每一个有收益数据的日期
        if date < np.datetime64('2023-07-01'):
            x_day=dff_ret[dff_ret['datetime']==date]
            r_day=0
            for instrument in larg_ins:  # 计算大收益股票的平均收益
                one_day = 0
                if len(x_day[x_day['instrument']==instrument]['close_r'].values):
                    one_day=x_day[x_day['instrument']==instrument]['close_r'].values[0]
                r_day=r_day+one_day
            for instrument in sma_ins:  # 从结果中减去小收益股票的平均收益
                r_day=r_day-1
            r_day=r_day/num  # 计算平均日收益
            b={'datetime':date,'daily_return':r_day}
            df_return=df_return.append(b,ignore_index=True)
    
    # 对于7月1日后的日期，重复上述过程
    day630=dff_ret[dff_ret['datetime']==np.datetime64('2023-06-31')]
    sma_ins=day630.nsmallest(num,columns='ret').instrument
    larg_ins=day630.nlargest(num,columns='ret').instrument
    for date in ret_date_unique:
        if date >= np.datetime64('2023-07-01'):
            x_day=dff_ret[dff_ret['datetime']==date]
            r_day=0
            for instrument in larg_ins:
                one_day = 0
                if len(x_day[x_day['instrument']==instrument]['close_r'].values):
                    one_day=x_day[x_day['instrument']==instrument]['close_r'].values[0]
                r_day=r_day+one_day
            for instrument in sma_ins:
                one_day = 0
                if len(x_day[x_day['instrument']==instrument]['close_r'].values):
                    one_day=x_day[x_day['instrument']==instrument]['close_r'].values[0]
                r_day=r_day-one_day
            r_day=r_day/num
            b={'datetime':date,'daily_return':r_day}
            df_return=df_return.append(b,ignore_index=True)

C:\Users\Liyt\AppData\Local\Temp\ipykernel_33872\1087805611.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tic['ret'][i]=(y-x)/x  # 计算收益率
d:\LenovoSoftstore\conda3\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3505: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Liyt\AppData\Local\Temp\ipykernel_33872\1087805611.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

ValueError: Day out of range in datetime string "2023-06-31"

In [140]:
if choice == 0:
    #反转策略
    dff_ret=pd.DataFrame()
    for ins in ins_unique:
        df_tic=df[df['instrument']==ins]
        df_tic.reset_index(inplace=True)
        for i in range(0,len(df_tic)):
            if df_tic['datetime'][i]>datetime(2023,1,1):
                date=df_tic['datetime'][i]
                loc=np.where(date_unique==date)
                predate_loc=np.where(date_unique==date)[0][0]-20
                date_pre=date_unique[predate_loc]
                x=df_tic.loc[df_tic['datetime']==date_pre]['close'].values[0]
                yestdaydate_loc=np.where(date_unique==date)[0][0]-1
                date_yest=date_unique[yestdaydate_loc]
                y=df_tic.loc[df_tic['datetime']==date_yest]['close'].values[0]
                df_tic['ret'][i]=(y-x)/x
        df_tic.dropna(how='any',inplace=True)
        dff_ret=pd.concat([dff_ret,df_tic],axis=0)
    dff_ret.reset_index(inplace=True)
    ret_date_unique=dff_ret['datetime'].unique()
    
    df_return=pd.DataFrame()
    
    day104=dff_ret[dff_ret['datetime']==np.datetime64('2023-01-03')]
    sma_ins=day104.nsmallest(9,columns='ret').instrument
    larg_ins=day104.nlargest(9,columns='ret').instrument
    for date in ret_date_unique:
        if date < np.datetime64('2023-08-01'):
            x_day=dff_ret[dff_ret['datetime']==date]
            r_day=0
            for instrument in larg_ins:
                r_day=r_day-1
            for instrument in sma_ins:
                one_day = 0
                if len(x_day[x_day['instrument']==instrument]['close_r'].values):
                    one_day=x_day[x_day['instrument']==instrument]['close_r'].values[0]
                r_day=r_day+one_day
            r_day=r_day/9
            b={'datetime':date,'daily_return':r_day}
            df_return=df_return._append(b,ignore_index=True)
            
    day630=dff_ret[dff_ret['datetime']==np.datetime64('2023-07-31')]
    sma_ins=day630.nsmallest(9,columns='ret').instrument
    larg_ins=day630.nlargest(9,columns='ret').instrument
    for date in ret_date_unique:
        if date >= np.datetime64('2023-08-01'):
            x_day=dff_ret[dff_ret['datetime']==date]
            r_day=0
            for instrument in larg_ins:
                one_day = 0
                if len(x_day[x_day['instrument']==instrument]['close_r'].values):
                    one_day=x_day[x_day['instrument']==instrument]['close_r'].values[0]
                r_day=r_day-one_day
            for instrument in sma_ins:
                one_day = 0
                if len(x_day[x_day['instrument']==instrument]['close_r'].values):
                    one_day=x_day[x_day['instrument']==instrument]['close_r'].values[0]
                r_day=r_day+one_day
            r_day=r_day/9
            b={'datetime':date,'daily_return':r_day}
            df_return=df_return._append(b,ignore_index=True)

In [157]:
df_return

,daily_return,datetime
0,0.020629,2023-01-03
1,0.011750,2023-01-04
2,0.020905,2023-01-05
3,-0.001986,2023-01-06
4,0.008500,2023-01-09
...,...,...
237,0.130800,2023-12-25
238,0.110339,2023-12-26
239,0.130201,2023-12-27
240,0.100544,2023-12-28


In [158]:
portfolio_df_performance = df_return.set_index(['datetime'])

In [159]:
portfolio_df_performance

,daily_return
datetime,
2023-01-03,0.020629
2023-01-04,0.011750
2023-01-05,0.020905
2023-01-06,-0.001986
2023-01-09,0.008500
...,...
2023-12-25,0.130800
2023-12-26,0.110339
2023-12-27,0.130201


In [160]:
ARR = (portfolio_df_performance['daily_return']+1).cumprod()
# ARR.to_csv("BLSW.csv", index=None)

In [161]:
ARR

datetime
2023-01-03    1.020629
2023-01-04    1.032620
2023-01-05    1.054207
2023-01-06    1.052113
2023-01-09    1.061056
                ...   
2023-12-25    3.944357
2023-12-26    4.379574
2023-12-27    4.949801
2023-12-28    5.447474
2023-12-29    6.201797
Name: daily_return, Length: 242, dtype: float64

In [162]:
alpha_df_performance = pd.DataFrame()
alpha_df_performance['portfolio_daily_return'] = portfolio_df_performance['daily_return']
alpha_df_performance['portfolio_net_value'] = (alpha_df_performance['portfolio_daily_return'] + 1).cumprod()

net_value_columns = ['portfolio_net_value']

alpha_statistics_df = pd.DataFrame(index=alpha_df_performance[net_value_columns].columns,
                                    columns=["年化收益", "年化波动率", "最大回撤率", "夏普率", "Calmar", "IR"])

# alpha_df_performance.set_index("dt", inplace=True)
alpha_df_performance.index = pd.to_datetime(alpha_df_performance.index)
monthly_statistics_df = alpha_df_performance[net_value_columns].resample('m').last()
monthly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    monthly_statistics_df])
monthly_statistics_df = monthly_statistics_df.pct_change()
monthly_statistics_df = monthly_statistics_df.dropna()
monthly_statistics_df.index = monthly_statistics_df.index.date
## TODO 补充第一年的数据
yearly_statistics_df = alpha_df_performance[net_value_columns].resample('y').last()
yearly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    yearly_statistics_df])
yearly_statistics_df = yearly_statistics_df.pct_change()
yearly_statistics_df = yearly_statistics_df.dropna()
yearly_statistics_df.index = yearly_statistics_df.index.date

alpha_statistics_df.loc[:, "年化收益"] = np.mean(
    (alpha_df_performance[net_value_columns].tail(1)) ** (252 / len(alpha_df_performance)) - 1)
alpha_statistics_df.loc[:, "年化波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252)
alpha_statistics_df.loc[:, "累积收益"] = np.mean(alpha_df_performance[net_value_columns].tail(1) - 1)
alpha_statistics_df.loc[:, "累积波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)
alpha_statistics_df.loc[:, "最大回撤率"] = np.min(
    (alpha_df_performance[net_value_columns] - alpha_df_performance[net_value_columns].cummax()) /
    alpha_df_performance[net_value_columns].cummax())
alpha_statistics_df.loc[:, "夏普率"] = alpha_statistics_df["年化收益"] / alpha_statistics_df["年化波动率"]
alpha_statistics_df.loc[:, "Calmar"] = alpha_statistics_df["年化收益"] / abs(alpha_statistics_df["最大回撤率"])
alpha_statistics_df.loc[:, "IR"] = np.mean(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252) / np.std(alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)

In [163]:
alpha_statistics_df

,年化收益,年化波动率,最大回撤率,夏普率,Calmar,IR,累积收益,累积波动率
portfolio_net_value,5.687537,0.515385,-0.216737,11.035506,26.241698,3.916189,5.201797,0.032466


BLSW（反转策略），CGM(动量策略)
ASR = ARR / AVol;   CR = ARR / abs(MDD)
              ARR    AVol    MDD    ASR    CR    IR
hs300  CGM  -0.265  0.204  -0.372 -1.298 -0.713 -1.439
       BLSW  0.127  0.195  -0.174  0.651  0.730  0.682
       
zz500  CGM   0.015  0.229  -0.179  0.066  0.084  0.001
       BLSW  0.110  0.227  -0.155  0.485  0.710  0.446
      
sp500  CGM   0.099  0.250  -0.139  0.396  0.712  0.584
       BLSW  0.199  0.318  -0.223  0.626  0.892  0.774
   
nas100 CGM   0.116  0.242  -0.145  0.479  0.800  0.603
       BLSW  0.368  0.339  -0.222  1.086  1.658  1.194

BLSW(反转策略), CGM(动量策略), AS(AlphaStock)
ASR = ARR / AVol;   CR = ARR / abs(MDD)
               ARR    AVol    MDD    ASR    CR    IR
data_1  CGM   0.014  0.198  -0.214  0.069  0.064  0.066
        BLSW -0.209  0.214  -0.341 -0.975 -0.613 -1.082
        AS   -0.121  0.116  -0.200 -1.046 -0.608 -1.062